# BigQuery Connection Guide

This notebook provides a step-by-step guide to connect to BigQuery datasets, including the Spider 2 dataset and your own custom datasets.

## Prerequisites

Before starting, ensure you have:

1. **Google Cloud Account**: Sign up at https://cloud.google.com/
2. **Google Cloud Project**: Create a project in the Google Cloud Console
3. **BigQuery API**: Enable the BigQuery API in your project
4. **Billing Account**: Link a billing account to your project (required for BigQuery)

### Cost Considerations
- BigQuery charges for query processing (first 1TB per month is free)
- Public datasets like Spider 2 are free to query
- Use the `dry_run` feature to estimate costs before running queries

## Step 1: Install Required Libraries

In [ ]:
# Install required libraries (run this if not already installed)
# !pip install google-cloud-bigquery google-auth pandas python-dotenv

import sys
import os
import json
import pandas as pd
from typing import Optional

# Add backend to path
sys.path.append(os.path.join(os.getcwd(), '..', 'backend'))

print("✓ Libraries imported successfully")

## Step 2: Set Up Authentication

### Method 1: Service Account (Recommended for Production)

1. **Create a Service Account**:
   - Go to Google Cloud Console → IAM & Admin → Service Accounts
   - Click "Create Service Account"
   - Give it a name and description
   - Grant roles: `BigQuery Data Viewer`, `BigQuery Job User`

2. **Download the JSON Key**:
   - Click on your service account
   - Go to "Keys" tab → "Add Key" → "Create new key" → JSON
   - Download and save the JSON file securely

3. **Set Environment Variable**:
   ```bash
   export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service-account-key.json"
   ```

In [ ]:
# Method 1: Using Service Account JSON file
SERVICE_ACCOUNT_PATH = "/path/to/your/service-account-key.json"  # Update this path
PROJECT_ID = "your-google-cloud-project-id"  # Update this

# Check if service account file exists
if os.path.exists(SERVICE_ACCOUNT_PATH):
    print(f"✓ Service account file found: {SERVICE_ACCOUNT_PATH}")
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_PATH
    os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
else:
    print(f"❌ Service account file not found: {SERVICE_ACCOUNT_PATH}")
    print("Please update the SERVICE_ACCOUNT_PATH variable above")

### Method 2: User Authentication (Good for Development)

1. **Install Google Cloud CLI**: https://cloud.google.com/sdk/docs/install
2. **Authenticate**: Run `gcloud auth application-default login`
3. **Set Project**: Run `gcloud config set project YOUR_PROJECT_ID`

In [ ]:
# Method 2: Using gcloud CLI authentication
# Uncomment and run this if you prefer user authentication

# import subprocess
# try:
#     # Check if gcloud is authenticated
#     result = subprocess.run(['gcloud', 'auth', 'list', '--filter=status:ACTIVE'], 
#                           capture_output=True, text=True)
#     if result.returncode == 0 and result.stdout.strip():
#         print("✓ gcloud authentication found")
#         print(result.stdout)
#     else:
#         print("❌ No active gcloud authentication")
#         print("Run: gcloud auth application-default login")
# except FileNotFoundError:
#     print("❌ gcloud CLI not found. Please install it first.")

### Method 3: Environment Variables with JSON Content

In [ ]:
# Method 3: Set credentials directly as environment variables
# This is useful for deployment environments like Docker, Heroku, etc.

# Example of setting credentials JSON directly
# GOOGLE_CREDENTIALS_JSON = '''
# {
#   "type": "service_account",
#   "project_id": "your-project-id",
#   "private_key_id": "...",
#   "private_key": "...",
#   "client_email": "...",
#   "client_id": "...",
#   "auth_uri": "https://accounts.google.com/o/oauth2/auth",
#   "token_uri": "https://oauth2.googleapis.com/token"
# }
# '''

# # Load from environment or .env file
# from dotenv import load_dotenv
# load_dotenv('../.env')

# credentials_json = os.getenv('GOOGLE_CREDENTIALS_JSON')
# if credentials_json:
#     print("✓ Found credentials in environment variables")
# else:
#     print("❌ No credentials found in environment variables")

print("Environment variables method ready (commented out for security)")

## Step 3: Test Basic BigQuery Connection

In [ ]:
# Import our BigQuery client
from data_connection.bigquery_client import BigQueryClient

# Test basic connection
try:
    # Initialize client
    client = BigQueryClient(
        project_id=PROJECT_ID,
        service_account_path=SERVICE_ACCOUNT_PATH if os.path.exists(SERVICE_ACCOUNT_PATH) else None
    )
    
    print(f"✓ Connected to BigQuery project: {client.project_id}")
    
    # Test with a simple query
    test_query = """
    SELECT 
        'Hello BigQuery!' as message,
        CURRENT_TIMESTAMP() as timestamp,
        @@version as version
    """
    
    print("\nTesting basic query...")
    result = client.query(test_query)
    print("✓ Basic query successful:")
    display(result)
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\nTroubleshooting tips:")
    print("1. Check your project ID")
    print("2. Verify BigQuery API is enabled")
    print("3. Confirm authentication method is set up correctly")
    print("4. Ensure billing is enabled for your project")

## Step 4: Explore Your Project's Datasets

In [ ]:
# List datasets in your project
try:
    from google.cloud import bigquery
    
    # List datasets
    datasets = list(client.client.list_datasets())
    
    if datasets:
        print(f"✓ Found {len(datasets)} datasets in project '{client.project_id}':")
        for dataset in datasets:
            print(f"  - {dataset.dataset_id}")
            
            # List tables in each dataset
            tables = list(client.client.list_tables(dataset.reference))
            if tables:
                print(f"    Tables ({len(tables)}): {[t.table_id for t in tables[:5]]}")
                if len(tables) > 5:
                    print(f"    ... and {len(tables) - 5} more")
            else:
                print("    No tables found")
            print()
    else:
        print(f"No datasets found in project '{client.project_id}'")
        print("You can create datasets in the BigQuery console or programmatically")
        
except Exception as e:
    print(f"❌ Failed to list datasets: {e}")

## Step 5: Access Public Datasets (Spider 2 Example)

In [ ]:
# Access the Spider 2 public dataset
try:
    print("Accessing Spider 2 public dataset...")
    
    # List Spider 2 tables
    spider_tables = client.list_spider2_tables()
    
    print(f"✓ Spider 2 dataset accessed successfully")
    print(f"Found {len(spider_tables)} tables")
    print("\nFirst 10 tables:")
    for table in spider_tables[:10]:
        print(f"  - {table}")
    
    if len(spider_tables) > 10:
        print(f"  ... and {len(spider_tables) - 10} more")
        
except Exception as e:
    print(f"❌ Failed to access Spider 2 dataset: {e}")
    print("Note: Public datasets should be accessible without special permissions")

## Step 6: Query Cost Estimation

In [ ]:
# Estimate query costs before running expensive queries
if 'spider_tables' in locals() and len(spider_tables) > 0:
    sample_table = spider_tables[0]
    
    # Create a sample query
    sample_query = f"""
    SELECT *
    FROM `spider2-public-data.spider2_1_0.{sample_table}`
    LIMIT 100
    """
    
    try:
        print(f"Estimating cost for query on table: {sample_table}")
        
        # Dry run to estimate cost
        cost_estimate = client.validate_query(sample_query)
        
        print("\n✓ Cost Estimation:")
        print(f"  Query is valid: {cost_estimate['valid']}")
        
        bytes_processed = cost_estimate.get('total_bytes_processed', 0)
        if bytes_processed:
            gb_processed = bytes_processed / (1024**3)
            estimated_cost = gb_processed * 5  # $5 per TB = $0.005 per GB
            
            print(f"  Bytes to process: {bytes_processed:,}")
            print(f"  GB to process: {gb_processed:.4f}")
            print(f"  Estimated cost: ${estimated_cost:.6f} USD")
        else:
            print("  Cost: Free (likely cached or very small)")
            
    except Exception as e:
        print(f"❌ Cost estimation failed: {e}")
else:
    print("⚠ No Spider 2 tables available for cost estimation")

## Step 7: Execute Sample Queries

In [ ]:
# Execute a safe, small query
if 'sample_table' in locals():
    try:
        print(f"Executing sample query on {sample_table}...")
        
        # Get sample data
        sample_data = client.sample_table_data(sample_table, limit=5)
        
        print(f"\n✓ Query executed successfully")
        print(f"Table: {sample_table}")
        print(f"Rows returned: {len(sample_data)}")
        print(f"Columns: {list(sample_data.columns)}")
        
        print("\nSample data:")
        display(sample_data)
        
    except Exception as e:
        print(f"❌ Query execution failed: {e}")
else:
    print("⚠ No table available for sample query")

## Step 8: Working with Your Own Data

In [ ]:
# Example: Create a dataset and table (optional)
# This is commented out to avoid creating resources accidentally

create_example = False  # Set to True if you want to create example data

if create_example:
    try:
        from google.cloud import bigquery
        
        dataset_id = f"{client.project_id}.example_dataset"
        
        # Create dataset
        dataset = bigquery.Dataset(dataset_id)
        dataset.location = "US"
        dataset = client.client.create_dataset(dataset, exists_ok=True)
        print(f"✓ Created dataset: {dataset.dataset_id}")
        
        # Create a simple table with sample data
        table_id = f"{dataset_id}.sample_table"
        
        # Define schema
        schema = [
            bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
            bigquery.SchemaField("name", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("email", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("created_at", "TIMESTAMP", mode="REQUIRED"),
        ]
        
        table = bigquery.Table(table_id, schema=schema)
        table = client.client.create_table(table, exists_ok=True)
        print(f"✓ Created table: {table.table_id}")
        
        # Insert sample data
        rows_to_insert = [
            {"id": 1, "name": "Alice", "email": "alice@example.com", "created_at": "2024-01-01 10:00:00"},
            {"id": 2, "name": "Bob", "email": "bob@example.com", "created_at": "2024-01-02 11:00:00"},
            {"id": 3, "name": "Charlie", "email": "charlie@example.com", "created_at": "2024-01-03 12:00:00"},
        ]
        
        errors = client.client.insert_rows_json(table, rows_to_insert)
        if errors == []:
            print("✓ Sample data inserted successfully")
        else:
            print(f"❌ Failed to insert data: {errors}")
            
    except Exception as e:
        print(f"❌ Failed to create example dataset: {e}")
else:
    print("Example dataset creation skipped (set create_example=True to enable)")

## Step 9: Advanced BigQuery Features

In [ ]:
# Demonstrate advanced BigQuery features

# 1. Parameterized queries (safer than string formatting)
def run_parameterized_query():
    try:
        from google.cloud import bigquery
        
        query = """
        SELECT 
            @message as greeting,
            @number as lucky_number,
            CURRENT_TIMESTAMP() as query_time
        """
        
        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("message", "STRING", "Hello from parameterized query!"),
                bigquery.ScalarQueryParameter("number", "INTEGER", 42),
            ]
        )
        
        query_job = client.client.query(query, job_config=job_config)
        result = query_job.result().to_dataframe()
        
        print("✓ Parameterized query executed:")
        display(result)
        
    except Exception as e:
        print(f"❌ Parameterized query failed: {e}")

# 2. Query with different output formats
def demonstrate_output_formats():
    try:
        simple_query = "SELECT 'test' as column1, 123 as column2"
        
        # As DataFrame
        df_result = client.query(simple_query, to_dataframe=True)
        print("DataFrame result:")
        print(type(df_result))
        display(df_result)
        
        # As list of dictionaries
        dict_result = client.query(simple_query, to_dataframe=False)
        print("\nDictionary result:")
        print(type(dict_result))
        print(dict_result)
        
    except Exception as e:
        print(f"❌ Output format demo failed: {e}")

print("Running advanced features demo...")
run_parameterized_query()
print("\n" + "="*50 + "\n")
demonstrate_output_formats()

## Step 10: Connection Management and Cleanup

In [ ]:
# Demonstrate proper connection management

# 1. Using context manager (recommended)
def demonstrate_context_manager():
    try:
        # This ensures automatic cleanup
        with BigQueryClient(project_id=PROJECT_ID) as temp_client:
            result = temp_client.query("SELECT 'Context manager test' as message")
            print("✓ Context manager query successful:")
            display(result)
        print("✓ Connection automatically closed")
        
    except Exception as e:
        print(f"❌ Context manager demo failed: {e}")

# 2. Manual cleanup
def demonstrate_manual_cleanup():
    temp_client = None
    try:
        temp_client = BigQueryClient(project_id=PROJECT_ID)
        result = temp_client.query("SELECT 'Manual cleanup test' as message")
        print("✓ Manual cleanup query successful:")
        display(result)
        
    except Exception as e:
        print(f"❌ Manual cleanup demo failed: {e}")
    finally:
        if temp_client:
            temp_client.close()
            print("✓ Connection manually closed")

print("Demonstrating connection management...")
# Note: Context manager may not work with our current implementation
# demonstrate_context_manager()
# print("\n" + "-"*30 + "\n")
demonstrate_manual_cleanup()

## Final Cleanup

In [ ]:
# Clean up the main client connection
try:
    if 'client' in locals():
        client.close()
        print("✓ Main BigQuery client connection closed")
except Exception as e:
    print(f"Warning: Error during cleanup: {e}")

print("\n🎉 BigQuery connection guide completed!")

## Summary and Next Steps

### What You've Learned:

1. **Authentication Methods**: Service account, user credentials, environment variables
2. **Basic Connections**: How to connect and test BigQuery access
3. **Dataset Exploration**: Listing datasets and tables
4. **Public Datasets**: Accessing Spider 2 and other public data
5. **Cost Management**: Estimating query costs before execution
6. **Query Execution**: Running queries and handling results
7. **Advanced Features**: Parameterized queries, output formats
8. **Best Practices**: Connection management and cleanup

### Troubleshooting Common Issues:

| Error | Solution |
|-------|----------|
| Authentication failed | Check service account permissions, verify JSON key |
| Project not found | Verify project ID, ensure project exists |
| BigQuery API not enabled | Enable BigQuery API in Google Cloud Console |
| Billing not enabled | Link a billing account to your project |
| Permission denied | Add BigQuery roles to your service account |
| Quota exceeded | Check your BigQuery quotas and limits |

### Next Steps:

1. **Explore Spider 2**: Use the `test_spider2_connection.ipynb` notebook
2. **Build Text-to-SQL**: Try the other notebooks for SQL generation
3. **Create Custom Datasets**: Upload your own data to BigQuery
4. **Optimize Queries**: Learn BigQuery best practices for performance
5. **Monitor Costs**: Set up billing alerts and query monitoring

### Useful Resources:

- [BigQuery Documentation](https://cloud.google.com/bigquery/docs)
- [BigQuery Python Client](https://googleapis.dev/python/bigquery/latest/)
- [BigQuery Pricing](https://cloud.google.com/bigquery/pricing)
- [BigQuery Public Datasets](https://cloud.google.com/bigquery/public-data)
- [Spider 2 Dataset](https://spider2-sql.github.io/)